In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.001

Inductance = 5.
R = 1.

n_func = 3
solver = "ca"
# solver = "ca_canonical"
# solver = "ca_canonical_X"

# Ciropt problem

In [3]:
num_fi = 3
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)
problem.obj = problem.b
res, opti = problem.solve(solver = solver, verbose=True)[:2]
ca_vars = problem.vars

# PEP verification

In [ ]:
res

{'b': -6.586851988799446e-12,
 'h': 2.0050490196829352e-11,
 'alpha': 0.00021484442616463077,
 'beta': 0.18371375512916682}

In [ ]:
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-9.141179865109008e-10
